# Evaluación de PC-SMOTE con Grid Search en el dataset Shuttle (Generación de caso base y datasets aumentados)


In [1]:
# lo que hace es modificar la lista de rutas de búsqueda de módulos de Python (sys.path) para incluir las carpetas ../scripts y ../datasets como ubicaciones adicionales donde Python puede buscar módulos o paquetes cuando hacés un import.
import sys
sys.path.append("../scripts")
sys.path.append("../datasets")

## Importación de módulos y librerías necesarias


In [2]:
# --- Módulos propios del proyecto ---
from cargar_dataset import cargar_dataset                      # Función para cargar datasets según configuración
from config_datasets import config_datasets                    # Diccionario de configuración de datasets
from evaluacion import evaluar_sampler_holdout                 # Evaluación de sobremuestreo con partición hold-out
from pc_smote import PCSMOTE                                   # Implementación principal de PCSMOTE
from graficador2d import Graficador2D                       # Clase para graficar resultados en 2D
from isolation_cleaner import IsolationCleaner                 # Clase para limpieza de outliers con Isolation Forest
from Utils import Utils                                       # Clase utilitaria con funciones auxiliares
from limpiador import LimpiadorOutliers                                 # Clase para limpieza de datos

from imblearn.over_sampling import SMOTE, BorderlineSMOTE, ADASYN

# --- Librerías estándar de Python ---
from datetime import datetime, timedelta                       # Manejo de fechas y tiempos
from itertools import product                                  # Generación de combinaciones de parámetros
import os                                                      # Operaciones con el sistema de archivos
from pathlib import Path

import traceback
# --- Librerías científicas ---
import numpy as np                                              # Operaciones numéricas y algebra lineal
import pandas as pd                                             # Manipulación y análisis de datos tabulares
from scipy.stats import uniform                                 # Distribuciones para búsqueda de hiperparámetros

# --- Scikit-learn: preprocesamiento ---
from sklearn.preprocessing import LabelEncoder, StandardScaler # Codificación de etiquetas y escalado de datos
from sklearn.pipeline import make_pipeline, Pipeline            # Creación de pipelines de procesamiento y modelado
from sklearn.preprocessing import RobustScaler

# --- Scikit-learn: división y validación ---
from sklearn.model_selection import (
    train_test_split,                                           # División de datos en train/test
    StratifiedKFold,                                            # Validación cruzada estratificada
    RandomizedSearchCV                                          # Búsqueda aleatoria de hiperparámetros
)

# --- Scikit-learn: reducción de dimensionalidad ---
from sklearn.decomposition import PCA                           # Análisis de Componentes Principales

# --- Scikit-learn: métricas ---
from sklearn.metrics import (
    f1_score,                                                    # Métrica F1-Score
    balanced_accuracy_score,                                     # Precisión balanceada
    matthews_corrcoef,                                           # Coeficiente MCC
    cohen_kappa_score,                                           # Kappa de Cohen
    make_scorer                                            
)

# --- Scikit-learn: clasificadores ---
from sklearn.ensemble import RandomForestClassifier             # Clasificador Random Forest
from sklearn.linear_model import LogisticRegression             # Regresión logística
from sklearn.svm import SVC                                      # Máquinas de Vectores de Soporte (SVM)

from sklearn.exceptions import ConvergenceWarning
import warnings

RANDOM_STATE = 42
RUTA_CLASICOS: str = "../datasets/datasets_aumentados/resampler_clasicos/",

## Generación del caso base

Este código realiza dos tareas principales para cada dataset configurado en `config_datasets`:

1. **Generar el caso base** (subcarpeta `datasets_aumentados/base/`):
   - Se crea un directorio específico para almacenar la versión original del dataset sin ningún tipo de sobremuestreo.
   - El dataset se carga utilizando la misma función `cargar_dataset` empleada en el pipeline principal.
   - Si las etiquetas (`y`) están en formato de texto u objeto, se convierten a valores numéricos con `LabelEncoder`.
   - Se realiza una división estratificada en conjuntos de entrenamiento y prueba (`train/test`) utilizando `train_test_split` con una proporción 70/30 y una semilla fija para asegurar reproducibilidad.
   - Se guardan dos archivos CSV: `<nombre_dataset>_train.csv` y `<nombre_dataset>_test.csv`.

In [3]:
def generar_caso_base(
    nombre_dataset: str,
    config: dict,
    ruta_base: str = "../datasets/datasets_aumentados/base/",
    test_size: float = 0.2,
    random_state: int = 42,
    overwrite: bool = False
):
    """
    Genera el caso base (sin sobremuestreo) aplicando RobustScaler
    y guardando train/test consistentes con el pipeline PCSMOTE.

    CORREGIDO:
      - El escalado se ajusta SOLO con X_train.
      - Luego se aplica el mismo scaler a X_test (sin fuga de información).
    """

    os.makedirs(ruta_base, exist_ok=True)

    path_train = os.path.join(ruta_base, f"{nombre_dataset}_train.csv")
    path_test  = os.path.join(ruta_base, f"{nombre_dataset}_test.csv")

    if not overwrite and os.path.exists(path_train) and os.path.exists(path_test):
        return path_train, path_test

    # 1) Cargar dataset original
    names = config.get("esquema") if config.get("header", None) is None else None

    X, y, _ = cargar_dataset(
        path=config.get("path"),
        clase_minoria=config.get("clase_minoria"),
        col_features=config.get("col_features"),
        col_target=config.get("col_target"),
        sep=config.get("sep", ","),
        header=config.get("header", None),
        binarizar=False,
        tipo=config.get("tipo", "tabular"),
        impute="median",
        names=names
    )

    # Asegurar DataFrame para tener nombres de columnas coherentes
    col_features = config.get("col_features")
    if not isinstance(X, pd.DataFrame):
        X = pd.DataFrame(X, columns=col_features)

    # 2) Codificar etiquetas si es necesario
    if getattr(y, "dtype", None) == object or (len(y) > 0 and isinstance(y[0], str)):
        y = LabelEncoder().fit_transform(y)

    # 3) Split estratificado sobre X crudo (sin escalar todavía)
    X_train_raw, X_test_raw, y_train, y_test = train_test_split(
        X, y, test_size=test_size, stratify=y, random_state=random_state
    )

    # 4) Ajustar RobustScaler SOLO con X_train y transformar ambos
    scaler = RobustScaler()
    X_train_scaled = scaler.fit_transform(X_train_raw)
    X_test_scaled  = scaler.transform(X_test_raw)

    # 5) Guardar CSVs consistentes con los aumentados
    df_train = pd.concat(
        [
            pd.DataFrame(X_train_scaled, columns=col_features),
            pd.Series(y_train, name=config.get("col_target", "target")),
        ],
        axis=1,
    )

    df_test = pd.concat(
        [
            pd.DataFrame(X_test_scaled, columns=col_features),
            pd.Series(y_test, name=config.get("col_target", "target")),
        ],
        axis=1,
    )

    df_train.to_csv(path_train, index=False)
    df_test.to_csv(path_test, index=False)

    print(f"✅ Caso base generado con escalado robusto para {nombre_dataset}")
    print(f"   Train: {path_train}")
    print(f"   Test : {path_test}")

    return path_train, path_test


In [4]:
def aumentar_dataset_pcsmote_y_guardar(
    nombre_dataset,
    X_train_base,
    y_train_base,
    percentil_densidad,
    percentil_riesgo,
    criterio_pureza,
    porcentaje_limpieza,
    col_target="target",
):
    try:
        # 1) Copias defensivas
        X = np.asarray(X_train_base)
        y = np.asarray(y_train_base).copy()
        n_total_original = len(y)

        # 2) Limpieza con IsolationCleaner
        if porcentaje_limpieza <= 0:
            X_limpio, y_limpio = X, y
            info_limpieza = {
                "percentil_umbral": 0.0,
                "removed_total": 0,
                "total": n_total_original,
            }
        else:
            X_limpio, y_limpio, info_raw = IsolationCleaner.limpiarOutliers(
                X=X,
                y=y,
                percentil_umbral=porcentaje_limpieza,
                devolver_info=True,
                verbose=False,
            )
            info_limpieza = {
                "percentil_umbral": info_raw["percentil_umbral"],
                "removed_total": info_raw["removed_total"],
                "total": n_total_original,
            }

        print(
            f"[{nombre_dataset}] I{porcentaje_limpieza}: "
            f"removidos={info_limpieza['removed_total']} / {info_limpieza['total']} "
            f"(p={info_limpieza['percentil_umbral']}%)"
        )

        # 3) PCSMOTE sobre el TRAIN limpio (ya escalado)
        sampler = PCSMOTE(
            random_state=42,
            percentil_densidad=percentil_densidad,
            percentil_entropia=75 if criterio_pureza == "entropia" else None,
            percentil_riesgo=percentil_riesgo,
            criterio_pureza=criterio_pureza,
            factor_equilibrio=1,
            metric="euclidean",
        )

        print(sampler.snapshot())

        if hasattr(sampler, "fit_resample_multiclass"):
            X_res, y_res = sampler.fit_resample_multiclass(X_limpio, y_limpio)
        else:
            X_res, y_res = sampler.fit_resample(X_limpio, y_limpio)

        # 4) Guardar TRAIN aumentado
        out_dir = Path("../datasets/datasets_aumentados/")
        out_dir.mkdir(parents=True, exist_ok=True)

        fname_train = (
            f"pcsmote_{nombre_dataset}_"
            f"D{percentil_densidad}_R{percentil_riesgo}_"
            f"{Utils.tag_p(criterio_pureza)}_I{porcentaje_limpieza}_train.csv"
        )
        fname_train = Utils.safe_token(fname_train)
        p_train = out_dir / fname_train

        df_train = pd.DataFrame(X_res)
        df_train[col_target] = y_res

        df_train.to_csv(p_train, index=False)

        return str(p_train), sampler

    except Exception as e:
        traceback.print_exc()
        print(f"❌ Error al aumentar dataset {nombre_dataset}: {e}")
        return None, None


In [5]:
def generar_aumentaciones_clasicas_y_guardar(
    nombre_dataset: str,
    X_train: pd.DataFrame,
    y_train: pd.Series,
    col_target: str,
    ruta_clasicos: str = "../datasets/datasets_aumentados/resampler_clasicos/",
    overwrite: bool = False,
):
    """
    Genera datasets aumentados con técnicas clásicas a partir de X_train, y_train:
      - SMOTE
      - BorderlineSMOTE
      - ADASYN

    Guarda los resultados en CSV en ruta_clasicos/ con nombres:
      smote_{dataset}_train.csv
      borderlinesmote_{dataset}_train.csv
      adasyn_{dataset}_train.csv
    """
    os.makedirs(ruta_clasicos, exist_ok=True)

    columnas = list(X_train.columns)

    print(f"🔧 Aumentación clásica (en memoria) para: {nombre_dataset}")
    print(f"   X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")

    resamplers = [
        ("smote", SMOTE(random_state=RANDOM_STATE)),
        ("borderlinesmote", BorderlineSMOTE(random_state=RANDOM_STATE, kind="borderline-1")),
        ("adasyn", ADASYN(random_state=RANDOM_STATE)),
    ]

    for nombre_resampler, resampler in resamplers:
        nombre_archivo = f"{nombre_resampler}_{nombre_dataset}_train.csv"
        path_salida = os.path.join(ruta_clasicos, nombre_archivo)

        if not overwrite and os.path.exists(path_salida):
            print(f"   ⚪ Omitido ({nombre_resampler}), ya existe: {nombre_archivo}")
            continue

        print(f"   🔁 Aplicando {nombre_resampler} ...")

        try:
            # fit_resample trabaja con ndarrays
            X_res, y_res = resampler.fit_resample(X_train.values, y_train.values)
        except ValueError as e:
            # Caso típico: ADASYN (u otro) decide no generar muestras
            print(f"   ⚠️ {nombre_resampler} no generó muestras sintéticas: {e}")
            print(f"      → Se omite guardar {nombre_resampler}_{nombre_dataset}_train.csv para este dataset.")
            continue

        df_res = pd.DataFrame(X_res, columns=columnas)
        df_res[col_target] = y_res  # y_res ya es array, no .values

        df_res.to_csv(path_salida, index=False)
        print(f"   ✅ Guardado: {path_salida}")


### 🧬 Aumento de Datasets mediante Técnicas de Sobremuestreo

En esta etapa se genera una versión balanceada de cada dataset original mediante la aplicación de técnicas de sobremuestreo, con el objetivo de mitigar el desbalance de clases antes del entrenamiento de los modelos.

Actualmente, se emplea la técnica:

- `PCSMOTE` (Percentile-Controlled SMOTE), que permite controlar la generación de muestras sintéticas en función de percentiles de densidad, riesgo y pureza.

Para cada dataset, se exploran combinaciones específicas de parámetros según la técnica utilizada. Los datasets resultantes se almacenan en el directorio `datasets/datasets_aumentados/`, utilizando nombres de archivo que reflejan la configuración empleada (por ejemplo: `pcsmote_nombre_D25_R50_Pentropia_train.csv`).

> ⚠️ Esta fase no incluye entrenamiento ni validación de modelos. Su único propósito es generar conjuntos de datos aumentados a partir del conjunto de entrenamiento. La partición `train/test` se realiza previamente, y **solo la parte de entrenamiento es sometida a sobremuestreo**. El conjunto de prueba permanece sin modificar para garantizar una evaluación imparcial posterior.


In [6]:
# --- GRID PCSMOTE ---
percentiles_densidad = [25, 50, 75]
percentiles_riesgo = [25, 50, 75]
criterios_pureza = ["entropia", "proporcion"]
percentil_isolation_cleaner = [0] #[0, 1, 5]

combinaciones = list(product(
    percentiles_densidad,
    percentiles_riesgo,
    criterios_pureza,
    percentil_isolation_cleaner
))

# --- Rutas base ---
os.makedirs("../logs/", exist_ok=True)

RUTA_CLASICOS = "../datasets/datasets_aumentados/resampler_clasicos/"
os.makedirs(RUTA_CLASICOS, exist_ok=True)

datasets_a_ignorar = {
    "eurosat",
    "shuttle",
    "glass",
    "heart",
    "wdbc",
    # "ecoli",
    "iris",
    "us_crime"
}

for nombre_dataset, config in config_datasets.items():
    if nombre_dataset in datasets_a_ignorar:
        continue

    print(f"\n📁 Dataset: {nombre_dataset}")

    # --- CASO BASE (única lectura del dataset crudo) ---
    base_train, base_test = generar_caso_base(nombre_dataset, config)
    print(f"🟦 Caso base generado:\n - Train: {base_train}\n - Test: {base_test}")

    # Cargar en memoria el TRAIN base ya escalado
    col_target = config.get("col_target", "target")
    df_base_train = pd.read_csv(base_train)

    if col_target not in df_base_train.columns:
        raise ValueError(f"La columna target '{col_target}' no está en {base_train}")

    # Versión DataFrame/Series para los clásicos
    X_train_df = df_base_train.drop(columns=[col_target])
    y_train_series = df_base_train[col_target]

    # Versión numpy para PCSMOTE (como ya lo usabas)
    X_train_base = X_train_df.values
    y_train_base = y_train_series.values

    # --- AUMENTACIÓN CLÁSICA (SMOTE, BorderlineSMOTE, ADASYN) ---
    generar_aumentaciones_clasicas_y_guardar(
        nombre_dataset=nombre_dataset,
        X_train=X_train_df,
        y_train=y_train_series,
        col_target=col_target,
        ruta_clasicos=RUTA_CLASICOS,
        overwrite=False,
    )

    # --- GRID DE PCSMOTE + ISOLATION CLEANER SOBRE ESTE TRAIN BASE ---
    for idx, (pdens, priesgo, criterio, plimpieza) in enumerate(combinaciones, start=1):
        print(
            f"#{idx:02d} ➕ Aumentando con "
            f"D={pdens} | R={priesgo} | P={criterio} | I={plimpieza}"
        )

        path_train, sampler = aumentar_dataset_pcsmote_y_guardar(
            nombre_dataset=nombre_dataset,
            X_train_base=X_train_base,
            y_train_base=y_train_base,
            percentil_densidad=pdens,
            percentil_riesgo=priesgo,
            criterio_pureza=criterio,
            porcentaje_limpieza=plimpieza,
            col_target=col_target,
        )

        sampler.nombre_dataset = nombre_dataset

        nombre_configuracion = f"D{int(pdens)}_R{int(priesgo)}" \
                            f"_{criterio}_I{plimpieza}"

        sampler.nombre_configuracion = nombre_configuracion

        if path_train and sampler:
            print(f"✅ Guardado exitoso:\n - Train: {path_train}")

            # --- rutas de logs (sanitizadas) ---
            base_logs = Path("../datasets/datasets_aumentados/logs/pcsmote")
            p_tag = Utils.tag_p(criterio)  # "Pentropia"/"Pproporcion"

            fname_clase = Utils.safe_token(
                f"log_pcsmote_{nombre_dataset}_D{pdens}_R{priesgo}_{p_tag}_I{plimpieza}.csv"
            )
            # Después (un solo archivo por dataset)
            fname_muestra = Utils.safe_token(
                f"log_pcsmote_x_muestra_{nombre_dataset}.xlsx"
            )


            log_path_x_clase = str(base_logs / "por_clase" / fname_clase)
            log_path_x_muestras = str(base_logs / "por_muestras" / fname_muestra)

            # Asegurar subdirectorios de logs
            os.makedirs(base_logs / "por_clase", exist_ok=True)
            os.makedirs(base_logs / "por_muestras", exist_ok=True)

            # --- export logs (CSV/Excel) ---
            sampler.exportar_log_csv(log_path_x_clase)
            sampler.exportar_log_muestras_excel(log_path_x_muestras)

            print(f"📄 Log exportado: {log_path_x_clase}")
            print(f"📄 Log exportado: {log_path_x_muestras}")
        else:
            print("❌ Falló la generación.")



📁 Dataset: ecoli
🟦 Caso base generado:
 - Train: ../datasets/datasets_aumentados/base/ecoli_train.csv
 - Test: ../datasets/datasets_aumentados/base/ecoli_test.csv
🔧 Aumentación clásica (en memoria) para: ecoli
   X_train shape: (268, 7), y_train shape: (268,)
   🔁 Aplicando smote ...
   ⚠️ smote no generó muestras sintéticas: Expected n_neighbors <= n_samples_fit, but n_neighbors = 6, n_samples_fit = 2, n_samples = 2
      → Se omite guardar smote_ecoli_train.csv para este dataset.
   🔁 Aplicando borderlinesmote ...
   ⚠️ borderlinesmote no generó muestras sintéticas: Expected n_neighbors <= n_samples_fit, but n_neighbors = 6, n_samples_fit = 4, n_samples = 3
      → Se omite guardar borderlinesmote_ecoli_train.csv para este dataset.
   🔁 Aplicando adasyn ...
   ⚠️ adasyn no generó muestras sintéticas: Expected n_neighbors <= n_samples_fit, but n_neighbors = 6, n_samples_fit = 2, n_samples = 2
      → Se omite guardar adasyn_ecoli_train.csv para este dataset.
#01 ➕ Aumentando con D=25

c:\Users\jnatello\Documents\GitHub\armado-tesina\codigo\notebooks\../scripts\pc_smote.py:170: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  return pd.concat(lista, ignore_index=True)


📄 Log exportado: ..\datasets\datasets_aumentados\logs\pcsmote\por_clase\log_pcsmote_ecoli_D25_R25_Pproporcion_I0.csv
📄 Log exportado: ..\datasets\datasets_aumentados\logs\pcsmote\por_muestras\log_pcsmote_x_muestra_ecoli.xlsx
#03 ➕ Aumentando con D=25 | R=50 | P=entropia | I=0
[ecoli] I0: removidos=0 / 268 (p=0.0%)
self.riesgo recibido en el construct:  50.0
{'k': 7, 'random_state': 42, 'percentil_dist': 75.0, 'percentil_densidad': 25.0, 'percentil_entropia': 75.0, 'percentil_riesgo': 50.0, 'criterio_pureza': 'entropia', 'factor_equilibrio': 1.0, 'metric': 'euclidean', 'guardar_distancias': True, 'max_total_multiplier': None, 'max_sinteticas_por_clase': None, 'nombre_dataset': 'unknown', 'nombre_configuracion': None}
self.riesgo recibido en el construct:  50.0
Umbral por percentil riesgo: 0.2857142857142857
self_meta: {'umbral_densidad': 0.42857142857142855, 'umbral_entropia': 0.8631205685666311, 'riesgo_medio': 0.4217687074829931, 'riesgo_std': 0.1426950813836941, 'densidad_media': 0.7

c:\Users\jnatello\Documents\GitHub\armado-tesina\codigo\notebooks\../scripts\pc_smote.py:170: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  return pd.concat(lista, ignore_index=True)


Umbral por percentil riesgo: 0.14285714285714285
self_meta: {'umbral_densidad': 0.5714285714285714, 'umbral_entropia': None, 'riesgo_medio': 0.2714285714285714, 'riesgo_std': 0.14914723584157927, 'densidad_media': 0.7589285714285714, 'vecinos_validos_promedio': 5.0, 'n_candidatas': 16, 'n_filtradas': 10, 'elapsed_ms': None, 'k_efectivo': 7, 'umbral_riesgo_min': 0.14285714285714285, 'umbral_densidad_global': 1.1003397984446452, 'pureza_eps': 0.14285714285714285, 'pureza_limite_inferior': 0.14285714285714285, 'pureza_limite_superior': 0.8571428571428572}
self.riesgo recibido en el construct:  50.0
self.riesgo recibido en el construct:  50.0
Umbral por percentil riesgo: 0.14285714285714285
self_meta: {'umbral_densidad': 0.5714285714285714, 'umbral_entropia': None, 'riesgo_medio': 0.261904761904762, 'riesgo_std': 0.12821820969367864, 'densidad_media': 0.7526132404181185, 'vecinos_validos_promedio': 5.0, 'n_candidatas': 41, 'n_filtradas': 18, 'elapsed_ms': None, 'k_efectivo': 7, 'umbral_rie

...

📄 Log exportado: ..\datasets\datasets_aumentados\logs\pcsmote\por_clase\log_pcsmote_ecoli_D75_R25_Pproporcion_I0.csv
📄 Log exportado: ..\datasets\datasets_aumentados\logs\pcsmote\por_muestras\log_pcsmote_x_muestra_ecoli.xlsx
#15 ➕ Aumentando con D=75 | R=50 | P=entropia | I=0
[ecoli] I0: removidos=0 / 268 (p=0.0%)
self.riesgo recibido en el construct:  50.0
{'k': 7, 'random_state': 42, 'percentil_dist': 75.0, 'percentil_densidad': 75.0, 'percentil_entropia': 75.0, 'percentil_riesgo': 50.0, 'criterio_pureza': 'entropia', 'factor_equilibrio': 1.0, 'metric': 'euclidean', 'guardar_distancias': True, 'max_total_multiplier': None, 'max_sinteticas_por_clase': None, 'nombre_dataset': 'unknown', 'nombre_configuracion': None}
self.riesgo recibido en el construct:  50.0
Umbral por percentil riesgo: 0.2857142857142857
self_meta: {'umbral_densidad': 1.0, 'umbral_entropia': 0.8631205685666311, 'riesgo_medio': 0.4117647058823529, 'riesgo_std': 0.1375689541491168, 'densidad_media': 0.7494145199063234,

c:\Users\jnatello\Documents\GitHub\armado-tesina\codigo\notebooks\../scripts\pc_smote.py:170: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  return pd.concat(lista, ignore_index=True)


📄 Log exportado: ..\datasets\datasets_aumentados\logs\pcsmote\por_clase\log_pcsmote_ecoli_D75_R50_Pentropia_I0.csv
📄 Log exportado: ..\datasets\datasets_aumentados\logs\pcsmote\por_muestras\log_pcsmote_x_muestra_ecoli.xlsx
#16 ➕ Aumentando con D=75 | R=50 | P=proporcion | I=0
[ecoli] I0: removidos=0 / 268 (p=0.0%)
self.riesgo recibido en el construct:  50.0
{'k': 7, 'random_state': 42, 'percentil_dist': 75.0, 'percentil_densidad': 75.0, 'percentil_entropia': None, 'percentil_riesgo': 50.0, 'criterio_pureza': 'proporcion', 'factor_equilibrio': 1.0, 'metric': 'euclidean', 'guardar_distancias': True, 'max_total_multiplier': None, 'max_sinteticas_por_clase': None, 'nombre_dataset': 'unknown', 'nombre_configuracion': None}
self.riesgo recibido en el construct:  50.0
Umbral por percentil riesgo: 0.2857142857142857
self_meta: {'umbral_densidad': 1.0, 'umbral_entropia': None, 'riesgo_medio': 0.4365079365079365, 'riesgo_std': 0.16817158809854835, 'densidad_media': 0.7494145199063234, 'vecinos_v

c:\Users\jnatello\Documents\GitHub\armado-tesina\codigo\notebooks\../scripts\pc_smote.py:170: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  return pd.concat(lista, ignore_index=True)


📄 Log exportado: ..\datasets\datasets_aumentados\logs\pcsmote\por_clase\log_pcsmote_ecoli_D75_R50_Pproporcion_I0.csv
📄 Log exportado: ..\datasets\datasets_aumentados\logs\pcsmote\por_muestras\log_pcsmote_x_muestra_ecoli.xlsx
#17 ➕ Aumentando con D=75 | R=75 | P=entropia | I=0
[ecoli] I0: removidos=0 / 268 (p=0.0%)
self.riesgo recibido en el construct:  75.0
{'k': 7, 'random_state': 42, 'percentil_dist': 75.0, 'percentil_densidad': 75.0, 'percentil_entropia': 75.0, 'percentil_riesgo': 75.0, 'criterio_pureza': 'entropia', 'factor_equilibrio': 1.0, 'metric': 'euclidean', 'guardar_distancias': True, 'max_total_multiplier': None, 'max_sinteticas_por_clase': None, 'nombre_dataset': 'unknown', 'nombre_configuracion': None}
self.riesgo recibido en el construct:  75.0
Umbral por percentil riesgo: 0.5714285714285714
self.riesgo recibido en el construct:  75.0
self.riesgo recibido en el construct:  75.0
self.riesgo recibido en el construct:  75.0
Umbral por percentil riesgo: 0.6071428571428571
se